### I have download my resume from linkedin in pdf format. I have also saved a short summary of myself into a file "summary.txt"
### I will use this as a resource/model context for the LLM and then will start a chat with LLM to ask questions about myself.

In [52]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr # A UI package

In [53]:
load_dotenv(override=True)
openai_python_client = OpenAI()


In [54]:
linkedin = ""
reader = PdfReader("linkedin.pdf")
for page in reader.pages:
    text = page.extract_text()
    linkedin += text    

In [ ]:
print(linkedin)

In [ ]:
with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

print(summary)

In [57]:
name = 'Rajat Girotra'

In [58]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [59]:
# This is a callback function which is called by the gradio whenever there is a message from the user/bot on the other side 
# of the chat. The argument is the latest message received and the history of the conversation.
def chat(message, history):
    # use the message and history to construct a prompt for openAI
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_python_client.chat.completions.create(model="gpt-4.1-nano", messages=messages)
    return response.choices[0].message.content

In [ ]:
# Now we will create a UI chat with a very simple api.
gr.ChatInterface(chat, type="messages").launch()

### In this next exercise, we will demonstrate the evaluator-optimizer workflow. We will show whether the response to a question is acceptable.
### We will show the evaluater LLM the latest response + the conversation history and it will evaluate if the latest response is acceptable.
### If not, we will provide the main LLM with the feedback provided by the evaluater LLM and ask the main LLM to generate a new response.

In [71]:
from pydantic import BaseModel

# just defining a model of the response expected from the evaluator LLM
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [72]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [73]:
def evaluator_user_prompt(reply, message, history=None):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{"No history for now" if history is None else history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [74]:
# Gemini will be our evaluator LLM
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [75]:
def evaluate(reply, message, history=None) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [65]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai_python_client.chat.completions.create(model="gpt-4.1-nano", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?")

In [78]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_python_client.chat.completions.create(model="gpt-4.1-nano", messages=messages)
    return response.choices[0].message.content

In [80]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai_python_client.chat.completions.create(model="gpt-4.1-nano", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()